<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tweet-data-import" data-toc-modified-id="Tweet-data-import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tweet data import</a></span><ul class="toc-item"><li><span><a href="#Small-subset-initial-import" data-toc-modified-id="Small-subset-initial-import-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Small subset initial import</a></span></li><li><span><a href="#Feature-selection" data-toc-modified-id="Feature-selection-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Feature selection</a></span></li><li><span><a href="#Full-datset-languages-check" data-toc-modified-id="Full-datset-languages-check-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Full datset languages check</a></span></li></ul></li><li><span><a href="#Write-out-data" data-toc-modified-id="Write-out-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Write out data</a></span></li></ul></div>

## Tweet data import

### Small subset initial import

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import preprocessor as tweet_preprocessor
pd.set_option('max_colwidth', None)
path = 'output_data/'

In [2]:
%%bash
head -n 1000 data/tweets.json > data/t.json

In [3]:
tweets_path = 'data/t.json'

%time df = pd.read_json(tweets_path, lines=True)
df.info()
columns = list(df.columns)

CPU times: user 28.4 ms, sys: 0 ns, total: 28.4 ms
Wall time: 29.1 ms
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   contributors               0 non-null      float64       
 1   coordinates                0 non-null      float64       
 2   created_at                 1000 non-null   datetime64[ns]
 3   display_text_range         1000 non-null   object        
 4   entities                   1000 non-null   object        
 5   favorite_count             1000 non-null   int64         
 6   favorited                  1000 non-null   bool          
 7   geo                        0 non-null      float64       
 8   id                         1000 non-null   int64         
 9   id_str                     1000 non-null   int64         
 10  in_reply_to_screen_name    44 non-null     object        
 11  

### Feature selection

In [4]:
unwanted = ['contributors', 'coordinates', 'geo', 'place']
columns = [x for x in columns if x not in unwanted]

In [5]:
df[columns].describe(include = bool)

,favorited,is_quote_status,retweeted,truncated
count,1000,1000,1000,1000
unique,1,1,1,1
top,False,False,False,False
freq,1000,1000,1000,1000


We need a function to get data from the 'entities' dictionary

In [6]:
def get_from_entities(row, type_field, value_field):
    field_values = []
    x = row['entities'][type_field]
    for i in range(len(x)):
        field_values.append(x[i][value_field])
    return field_values

def get_hashtags(row):
    values = get_from_entities(row,
                               type_field='hashtags',
                               value_field='text')
    hashtags.update(values)
    for h in values:
        tweets_hashtags.append([row['id'],h])
    return values

def get_mentions(row):
    values = get_from_entities(row,
                               type_field='user_mentions',
                               value_field='screen_name')
    mentions.update(values)
    for h in values:
        tweets_mentions.append([row['id'],h])
    return values

In [7]:
df['hashtags'] = df.apply(get_from_entities, axis=1,
                          type_field='hashtags',
                          value_field='text')
df['mentions'] = df.apply(get_from_entities, axis=1,
                           type_field='user_mentions',
                           value_field='screen_name')

In [8]:
# The twitter client might be useful in another context but not here
# I have no need for display_text_range when the text is available
# Remove 'entities' now that I have hashtags and mentions
unwanted = ['favorited', 'is_quote_status','retweeted','truncated',
            'entities', 'source', 'display_text_range']
columns = [x for x in columns if x not in unwanted]

In [9]:
df[columns].describe(include=int)

,favorite_count,id,id_str,retweet_count,user_id
count,1000.000000,1.000000e+03,1.000000e+03,1000.000000,1.000000e+03
mean,0.142000,1.231684e+09,1.231684e+09,0.002000,1.705780e+07
std,0.556908,1.139083e+08,1.139083e+08,0.044699,4.346137e+06
min,0.000000,8.774186e+08,8.774186e+08,0.000000,5.558312e+06
25%,0.000000,1.170882e+09,1.170882e+09,0.000000,1.595500e+07
50%,0.000000,1.247574e+09,1.247574e+09,0.000000,1.816678e+07
75%,0.000000,1.318437e+09,1.318437e+09,0.000000,1.941846e+07
max,8.000000,1.402545e+09,1.402545e+09,1.000000,2.677811e+07


In [10]:
# Few tweets are 'liked' so lose 'favorite_count'
unwanted = ['id_str', 'favorite_count']
columns = [x for x in columns if x not in unwanted]

In [11]:
df[columns].describe(include=np.object)

,in_reply_to_screen_name,lang,screen_name,text
count,44,1000,1000,1000
unique,40,7,31,994
top,Citizensnews,en,henrymcmaster,Attending a Whip Team meeting
freq,2,986,117,2


Both screen_name and user_id are unique to the tweeter.
I'll lose user_id and match/join on screen_name if necessary.

In [12]:
unwanted = ['user_id']
columns = [x for x in columns if x not in unwanted]

In [13]:
df[columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   created_at                 1000 non-null   datetime64[ns]
 1   id                         1000 non-null   int64         
 2   in_reply_to_screen_name    44 non-null     object        
 3   in_reply_to_status_id      37 non-null     float64       
 4   in_reply_to_status_id_str  37 non-null     float64       
 5   in_reply_to_user_id        44 non-null     float64       
 6   in_reply_to_user_id_str    44 non-null     float64       
 7   lang                       1000 non-null   object        
 8   retweet_count              1000 non-null   int64         
 9   screen_name                1000 non-null   object        
 10  text                       1000 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 86

Replies are < 4% of the data so ignore `in_reply_to_*`.
Also lose the `lang` column but it will be used briefly before the cleaned
*full* data set is saved.

In [14]:
unwanted = ['in_reply_to_status_id_str',
            'in_reply_to_status_id',
            'in_reply_to_user_id_str',
            'in_reply_to_user_id',
            'in_reply_to_screen_name',
            'lang']
columns = [x for x in columns if x not in unwanted]

In [15]:
df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'favorite_count', 'favorited', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'retweet_count', 'retweeted', 'screen_name', 'source', 'text',
       'truncated', 'user_id', 'hashtags', 'mentions'],
      dtype='object')

### Full datset languages check

Should be able to store just what's required from the full tweets data file.

In [16]:
tweets_path = 'data/tweets.json'
chunksize = 10000

Lets get a full list of languages

In [17]:
%%time

def count_languages(chunk, languages):
    for lang in chunk['lang']:
        if lang not in languages:
            languages[lang] = 0
        languages[lang] += 1
        
# This takes a while but at least it's comprehensive
# Remember Twitter *autodetects* the tweet's language
# so chances are most 'und' is from English speakers.
languages = dict()
for chunk in pd.read_json(tweets_path, lines=True, chunksize=chunksize):
    count_languages(chunk, languages)
print (languages)

{'en': 1226949, 'nl': 88, 'es': 5108, 'fr': 674, 'und': 8137, 'sv': 57, 'ro': 261, 'is': 17, 'tl': 245, 'ht': 107, 'in': 346, 'de': 223, 'et': 205, 'pl': 183, 'no': 65, 'da': 121, 'fi': 88, 'sl': 23, 'it': 120, 'cy': 74, 'cs': 15, 'eu': 19, 'lv': 13, 'pt': 142, 'vi': 11, 'th': 1, 'hi': 13, 'hu': 9, 'tr': 16, 'lt': 23, 'zh': 1, 'uk': 5, 'ko': 1, 'ja': 3, 'iw': 3, 'ru': 2, 'fa': 2}
CPU times: user 56.6 s, sys: 629 ms, total: 57.3 s
Wall time: 57.8 s


Almost 98.7% of the tweets are in English so I'll lose tweets where
`lang != 'en'` as well as the `lang` column itself.

Add the hashtags and mentions columns too, before writing to csv.
Sadly parquet et al. do not support appending data.
(HDF does but I don't have time to learn enough about it.)

## Write out data

In [18]:
# Keep a list of tweeters' screen names, mentions and hashtags
# These will all need to have tables (see ERD)
hashtags = set() # Global
mentions = set() # Global
tweets_hashtags = [] # Global
tweets_mentions = [] # Global

In [19]:
%%time
# This builds up data needed for the hashtag, mention, tweet_hashtag and
# tweet_mention relations as well as the main tweets table.
# It takes a while to run!
outpath = path+'tweet.csv'

#Create "tweets" table headers
df[df['id']==None][columns].to_csv(outpath,
                                   mode='w',
                                   header=True,
                                   index=False,
                                   line_terminator='r')

for chunk in pd.read_json(tweets_path, lines=True, chunksize=chunksize):
    # Lose non-English tweets
    chunk = chunk[chunk['lang'] == 'en']
    
    # Add hashtags and mentions from each tweet
    # These functions access global variables to create
    # the four tables mentioned above.
    chunk['hashtags'] = chunk.apply(get_hashtags, axis=1)
    chunk['mentions'] = chunk.apply(get_mentions, axis=1)
    chunk['text'] = chunk['text'].apply(tweet_preprocessor.clean)
    
    # I'm not going to refactor the chunk processing but I will
    # no longer need the 'hashtags' nor 'mentions' columns
    unwanted = ['hashtags', 'mentions']
    columns = [x for x in columns if x not in unwanted]
    
    # Select columns
    chunk = chunk[columns]

    # Write to CSV
    chunk.to_csv(outpath,
                 mode='a',
                 header=False,
                 index=False,
                 line_terminator='r')

print(f'Output columns: {columns}')

Output columns: ['created_at', 'id', 'retweet_count', 'screen_name', 'text']
CPU times: user 3min 39s, sys: 731 ms, total: 3min 39s
Wall time: 3min 42s


In [20]:
chunk.columns

Index(['created_at', 'id', 'retweet_count', 'screen_name', 'text'], dtype='object')

And these columns should match the ERD (for now).

In [21]:
# Create the secondary tables
hashtags = pd.DataFrame(hashtags, columns=['hashtag'], dtype=str)
mentions = pd.DataFrame(mentions, columns=['mention'], dtype=str)
tweets_hashtags = pd.DataFrame(tweets_hashtags,columns=['tweet_id','hashtag'])
tweets_mentions = pd.DataFrame(tweets_mentions,columns=['tweet_id','mention'])

In [22]:
# Summarise scondary table sizes
print('Sizes of secondary tables')
print('-------------------------')
print(f'hashtag: {hashtags.shape}')
print(f'mention: {mentions.shape}')
print(f'tweet_hashtag: {tweets_hashtags.shape}')
print(f'tweet_mention: {tweets_mentions.shape}')

Sizes of secondary tables
-------------------------
hashtag: (96973, 1)
mention: (132149, 1)
tweet_hashtag: (893921, 2)
tweet_mention: (998478, 2)


In [23]:
# Output secondary tables
hashtags.to_csv(path+'hashtag.csv', index=False)
mentions.to_csv(path+'mention.csv', index=False)
tweets_hashtags.to_csv(path+'tweet_hashtag.csv', index=False)
tweets_mentions.to_csv(path+'tweet_mention.csv', index=False)